<a href="https://colab.research.google.com/github/jongkwangyun/quant/blob/main/VAA_aggresive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

In [ ]:
pip install pandas_market_calendars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.4/191.4 kB 9.2 MB/s eta 0:00:00


In [ ]:
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()
import pandas as pd
import exchange_calendars as ecals
from datetime import datetime, timedelta

pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.max_columns', None)

def get_price_data(x, start_date, end_date):
    df_price = pd.DataFrame(columns=x)
    for ticker in x:
        df_price[ticker] = pdr.get_data_yahoo(ticker, start=start_date, end=end_date)['Adj Close']
    return df_price.dropna(axis=0)

# 1년 전 개장일 구하기
us_market_code = "XNYS"
us_calendar = ecals.get_calendar(us_market_code)

today = datetime.today().date()
today_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=0)).date())
month_01_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=30)).date())
month_03_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=91)).date())
month_06_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=182)).date())
month_12_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=365)).date())

VAA_agg = ['SPY', 'EFA', 'EEM', 'AGG', 'LQD', 'IEF', 'SHY']

df_VAA_agg = get_price_data(VAA_agg, month_12_ago_workday, today).sort_index(ascending=False)
display(df_VAA_agg)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,SPY,EFA,EEM,AGG,LQD,IEF,SHY
Date,,,,,,,
2023-08-07,450.7100,72.7700,40.4800,96.7000,105.6400,94.8200,81.1000
2023-08-04,446.8100,72.1600,40.5400,96.6800,105.9000,95.0500,81.0700
2023-08-03,448.8400,71.9600,40.5900,95.8900,104.8100,94.0500,80.9100
2023-08-02,450.1300,72.1600,40.4300,96.5400,105.7600,94.7500,80.8800
2023-08-01,456.4800,73.5300,41.4200,96.7700,106.4100,94.9400,80.8400
...,...,...,...,...,...,...,...
2022-08-11,413.2419,64.7976,39.6992,99.7360,108.6796,100.6321,80.5230
2022-08-10,413.2419,64.8172,39.5428,100.2703,109.7383,101.3046,80.5522
2022-08-09,404.7407,63.3892,39.0541,100.0177,108.7085,101.2461,80.4742


In [ ]:
'''
spy_today_price = df_VAA_agg.loc[today_workday, 'SPY']
spy_month_01_ago_price = df_VAA_agg.loc[month_01_ago_workday, 'SPY']
spy_month_03_ago_price = df_VAA_agg.loc[month_03_ago_workday, 'SPY']
spy_month_06_ago_price = df_VAA_agg.loc[month_06_ago_workday, 'SPY']
spy_month_12_ago_price = df_VAA_agg.loc[month_12_ago_workday, 'SPY']

spy_month_01_ror = (spy_today_price/spy_month_01_ago_price - 1)*100
spy_month_03_ror = (spy_today_price/spy_month_03_ago_price - 1)*100
spy_month_06_ror = (spy_today_price/spy_month_06_ago_price - 1)*100
spy_month_12_ror = (spy_today_price/spy_month_12_ago_price - 1)*100

spy_momentum_score = 12 * spy_month_01_ror + 4 * spy_month_03_ror + 2 * spy_month_06_ror + 1 * spy_month_12_ror

print(spy_momentum_score)
'''

104.08988781551452


In [ ]:
ago = ['today', 'month_01_ago', 'month_03_ago', 'month_06_ago', 'month_12_ago']
etf_data = {}

for i in range(7):
  etf_data[VAA_agg[i]] = [df_VAA_agg.loc[today_workday, VAA_agg[i]], df_VAA_agg.loc[month_01_ago_workday, VAA_agg[i]], df_VAA_agg.loc[month_03_ago_workday, VAA_agg[i]], df_VAA_agg.loc[month_06_ago_workday, VAA_agg[i]], df_VAA_agg.loc[month_12_ago_workday, VAA_agg[i]]]

etf = pd.DataFrame(etf_data, columns=VAA_agg, index=ago)
display(etf)

,SPY,EFA,EEM,AGG,LQD,IEF,SHY
today,450.7100,72.7700,40.4800,96.7000,105.6400,94.8200,81.1000
month_01_ago,438.5500,70.9000,39.3900,96.2681,105.1789,94.3509,80.5888
month_03_ago,411.2125,72.5697,39.1037,98.3158,106.6986,98.3589,81.4605
month_06_ago,406.7609,69.4175,39.9669,97.7551,107.5273,96.4169,80.1323
month_12_ago,406.8266,63.5848,39.1519,99.7845,108.7663,100.9538,80.4742


In [ ]:
etf_momentum_score = []

for etf_name in VAA_agg:
  etf_today_price = etf.loc[ago[0], etf_name]
  etf_momentum_score.append((12 * (etf_today_price/etf.loc[ago[1], etf_name] - 1) * 100) + (4 * (etf_today_price/etf.loc[ago[2], etf_name] - 1) * 100) + (2 * (etf_today_price/etf.loc[ago[3], etf_name] - 1) * 100) + (1 * (etf_today_price/etf.loc[ago[4], etf_name] - 1) * 100))

display(pd.DataFrame(etf_momentum_score, columns=['momentum_score'], index=VAA_agg).transpose())

,SPY,EFA,EEM,AGG,LQD,IEF,SHY
momentum_score,104.0899,56.8583,53.2453,-6.4405,-5.0924,-17.8141,9.0339


In [ ]:
# 0 이하인 값이 있는지 확인
has_negative_values = any(score <= 0 for score in etf_momentum_score[:4])

if has_negative_values:
    print("공격형 자산 중 0 이하 발생")
    print("추천 안전 자산: ", VAA_agg[etf_momentum_score.index(max(etf_momentum_score[4:]))])
else:
    print("공격형 자산 모두 0 이상")
    print("추천 공격 자산: ", VAA_agg[etf_momentum_score.index(max(etf_momentum_score[:4]))])


공격형 자산 중 0 이하 발생
추천 안전 자산:  SHY
